# Notebook 4-1
# Models ensembling, evaluation

* Objective of this notebook is:
* Get best model available with  predictive features, that we found after filltering insignificant features and found best set of features for LIGHT Gradient Boosting model by implementing recursive faeture addition.

In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
from skopt import BayesSearchCV
from feature_engine.selection import RecursiveFeatureAddition
import shap
from collections import Counter
import warnings
import scipy
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, average_precision_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
import shap
import joblib
from sklearn import metrics
from scipy.stats import mannwhitneyu, chi2_contingency
from sklearn.metrics import make_scorer, silhouette_score
from sklearn.model_selection import StratifiedKFold,KFold,RandomizedSearchCV
import lightgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import sweetviz as sv
from IPython.display import display, IFrame,Image
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
import seaborn as sns
import analysis_functions
from IPython.display import display, IFrame
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    recall_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    balanced_accuracy_score,
    f1_score,
)
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")


%matplotlib inline

In [2]:
df_modeling = pd.read_csv("prepared_data/post_modelling_RFA_data.csv")
df_modeling

,SK_ID_CURR,AMT_CREDIT,DAYS_BIRTH,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,DAYS_CREDIT_Bureau,TARGET
0,100002,406597.5,-9461,Cash loans,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,-874.000000,1
1,100003,1293502.5,-16765,Cash loans,F,Family,Other,Higher education,Married,House / apartment,Core staff,MONDAY,Other,-1400.750000,0
2,100004,135000.0,-19046,Revolving loans,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Other,-867.000000,0
3,100006,312682.5,-19005,Cash loans,F,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,0
4,100007,513000.0,-19932,Cash loans,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Other,-1149.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,254700.0,-9327,Cash loans,M,Unaccompanied,Working,Secondary / secondary special,Other,With parents,Sales staff,THURSDAY,Other,NaN,0
307507,456252,269550.0,-20775,Cash loans,F,Unaccompanied,Pensioner,Secondary / secondary special,Other,House / apartment,Other,MONDAY,XNA,NaN,0
307508,456253,677664.0,-14966,Cash loans,F,Unaccompanied,Working,Higher education,Other,House / apartment,Managers,THURSDAY,Other,-867.500000,0
307509,456254,370107.0,-11961,Cash loans,F,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,WEDNESDAY,Other,-1104.000000,1


In [3]:
classification_setup = setup(
    df_modeling,
    target="TARGET",
    normalize=True, 
    normalize_method='zscore',
    transformation=True,
    train_size = 0.9,
    session_id=42,
    fix_imbalance=True,
    fix_imbalance_method='randomundersampler'
    
)

,Description,Value
0,Session id,42
1,Target,TARGET
2,Target type,Binary
3,Original data shape,"(307511, 15)"
4,Transformed data shape,"(75436, 44)"
5,Transformed train set shape,"(44684, 44)"
6,Transformed test set shape,"(30752, 44)"
7,Ordinal features,1
8,Numeric features,4
9,Categorical features,10


In [4]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.9193,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,2.5020
lightgbm,Light Gradient Boosting Machine,0.6206,0.6619,0.6137,0.1245,0.2071,0.0841,0.1307,3.1930
catboost,CatBoost Classifier,0.6169,0.6613,0.6179,0.1241,0.2066,0.0834,0.1304,15.1370
gbc,Gradient Boosting Classifier,0.6156,0.6631,0.6238,0.1245,0.2076,0.0844,0.1325,7.0610
qda,Quadratic Discriminant Analysis,0.6117,0.5111,0.3783,0.0854,0.1219,0.0060,0.0079,2.5910
lr,Logistic Regression,0.6110,0.6492,0.6082,0.1208,0.2015,0.0772,0.1217,4.0660
ridge,Ridge Classifier,0.6105,0.0000,0.6084,0.1207,0.2014,0.0770,0.1214,2.5660
lda,Linear Discriminant Analysis,0.6105,0.6492,0.6084,0.1207,0.2014,0.0770,0.1214,2.5300
ada,Ada Boost Classifier,0.6101,0.6589,0.6261,0.1232,0.2059,0.0821,0.1301,4.0030
rf,Random Forest Classifier,0.6024,0.6311,0.5874,0.1152,0.1926,0.0666,0.1058,6.4570


In [5]:
feat_to_drop=['TARGET','SK_ID_CURR']

In [6]:
categorical_columns = df_modeling.select_dtypes(include=['object']).columns

df_modeling[categorical_columns] = df_modeling[categorical_columns].astype('category')

In [18]:
X.columns.tolist()

['AMT_CREDIT',
 'DAYS_BIRTH',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'DAYS_CREDIT_Bureau']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,stratify=y, random_state=42)

In [9]:
np.int = int
np.float = float
np.bool = bool

In [10]:
xgb_param_grid = {
    'boosting_type': ['dart'],
    'learning_rate': (0.01, 0.1),  
    'n_estimators': (150, 900),
    'max_depth': (5, 10),
    'scale_pos_weight': (11,),
    'reg_lambda': (0.5, 10),
    'num_leaves': (30,900)
}

xgb_model = XGBClassifier(enable_categorical=True)

xgb_bayes_search = BayesSearchCV(xgb_model, xgb_param_grid, n_iter=50, scoring='roc_auc', cv=3, verbose=1, n_jobs=-1)
xgb_bayes_search.fit(X_train, y_train)

# Print best hyperparameters for XGBClassifier
print("\nBest Hyperparameters for XGBClassifier:")
print(xgb_bayes_search.best_params_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

In [11]:
xgb_bparams = xgb_bayes_search.best_params_

In [12]:
lgbm_param_grid = {
    'learning_rate': (0.01, 0.1),
    'n_estimators': (150, 900),
    'max_depth': (5, 10),
    'num_leaves': (30,900),
    'scale_pos_weight': (11,),
    'reg_lambda': (0.5, 5),
    "bagging_freq": (5, 20),
    "bagging_fraction": (0.7, 0.9)
}

lgbm_model = LGBMClassifier()
lgbm_bayes_search = BayesSearchCV(lgbm_model, lgbm_param_grid, n_iter=50, scoring='roc_auc', cv=3, verbose=1, n_jobs=-1)
lgbm_bayes_search.fit(X_train, y_train)

print("Best Hyperparameters for LGBMClassifier:")
print(lgbm_bayes_search.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

  File "C:\Users\artur\anaconda3\envs\Stroke\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Info] Number of positive: 17377, number of negative: 197880
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015067 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 814
[LightGBM] [Info] Number of data points in the train set: 215257, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080727 -> i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [13]:
Lgbm_bparams = lgbm_bayes_search.best_params_

In [14]:
print("Best Hyperparameters for LGBMClassifier:")
print(lgbm_bayes_search.best_params_)

Best Hyperparameters for LGBMClassifier:
OrderedDict([('bagging_fraction', 0.7907438990726503), ('bagging_freq', 10), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 900), ('num_leaves', 30), ('reg_lambda', 0.521428822889528), ('scale_pos_weight', 11)])


In [15]:
categorical_columns = X.select_dtypes(include=['object']).columns
X[categorical_columns] = X[categorical_columns].astype('category')

n_splits = 5
random_state = 22
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

models = [
    LGBMClassifier(
       **Lgbm_bparams
    ),
    XGBClassifier(
        **xgb_bparams, enable_categorical=True
    ),
]

soft_voting_classifier = VotingClassifier(
    estimators=[
        ('lgbm', LGBMClassifier(
            **Lgbm_bparams
        )),
        ('xgb', XGBClassifier(
            **xgb_bparams, enable_categorical=True
        )),
    ],
    voting='soft'
)

classifiers = [
    "LGB",
    "XGB",
    "SOFTVOTE",
]

recall_macro_scores = []  
auc_pr_scores = []  

for model, classifier_name in zip(models + [soft_voting_classifier], classifiers):
    cv_result_auc_pr = cross_val_score(model, X, y, cv=kfold, scoring="roc_auc")
    auc_pr_scores.append(cv_result_auc_pr.mean())

    recall_macro_scores_list = []
    auc_pr_scores_list = []
    
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        recall_macro_scores_list.append(recall_score(y_test, (y_pred_proba > 0.5).astype(int), average='macro'))
        auc_pr_scores_list.append(roc_auc_score(y_test, y_pred_proba, average='weighted'))

    recall_macro_scores.append(np.mean(recall_macro_scores_list))

# Create a DataFrame to display the results
new_models_dataframe2 = pd.DataFrame(
    {"CV Recall Macro Mean": recall_macro_scores, "Recall Macro Mean": recall_macro_scores,
     "CV AUC-PR Mean": auc_pr_scores, "AUC-PR Mean": auc_pr_scores}, index=classifiers
)
new_models_dataframe2

[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Info] Number of positive: 19988, number of negative: 226020
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019841 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 814
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081249 -> i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


,CV Recall Macro Mean,Recall Macro Mean,CV AUC-PR Mean,AUC-PR Mean
LGB,0.622764,0.622764,0.668401,0.668401
XGB,0.621320,0.621320,0.666085,0.666085
SOFTVOTE,0.622528,0.622528,0.667812,0.667812


In [16]:
xgb_model = XGBClassifier(**xgb_bparams, enable_categorical=True)
xgb_predictions = cross_val_predict(xgb_model, X, y, cv=kfold)

# Add XGBoost predictions as an extra feature to the original dataset
X_with_xgb_preds = pd.concat([X, pd.Series(xgb_predictions, name='xgb_preds')], axis=1)

# Fit LGBM model using cross-validation
lgb_model = LGBMClassifier(**Lgbm_bparams)
lgb_predictions = cross_val_predict(lgb_model, X_with_xgb_preds, y, cv=kfold)

# Calculate ROC-AUC score for LGBM predictions
roc_auc = roc_auc_score(y, lgb_predictions)
print(f"ROC-AUC Score for LGBM: {roc_auc}")

# Print classification report and confusion matrix for XGBoost
print("Classification Report for XGBoost:")
print(classification_report(y, xgb_predictions))

print("Confusion Matrix for XGBoost:")
print(confusion_matrix(y, xgb_predictions))

# Print classification report and confusion matrix for LightGBM
print("Classification Report for LightGBM:")
print(classification_report(y, lgb_predictions))

print("Confusion Matrix for LightGBM:")
print(confusion_matrix(y, lgb_predictions))

[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] bagging_fraction is set=0.7907438990726503, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7907438990726503
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Info] Number of positive: 19988, number of negative: 226020
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022388 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 816
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081249 -> i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Exception ignored on calling ctypes callback function: <function _log_callback at 0x000001D95D1D1120>
Traceback (most recent call last):
  File "C:\Users\artur\anaconda3\envs\Stroke\Lib\site-packages\lightgbm\basic.py", line 203, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

In [ ]:
xgb_model = XGBClassifier(**xgb_bparams, enable_categorical=True)
xgb_probabilities = cross_val_predict(xgb_model, X, y, cv=kfold, method='predict_proba')[:, 1]

# Add XGBoost probabilities as an extra feature to the original dataset
X_with_xgb_probs = pd.concat([X, pd.Series(xgb_probabilities, name='xgb_probs')], axis=1)

# Fit LGBM model using cross-validation
lgb_model = LGBMClassifier(**Lgbm_bparams)
lgb_predictions = cross_val_predict(lgb_model, X_with_xgb_probs, y, cv=kfold)

# Calculate ROC-AUC score for LGBM predictions
roc_auc = roc_auc_score(y, lgb_predictions)
print(f"ROC-AUC Score for LGBM: {roc_auc}")

In [ ]:
roc_auc_xgb = roc_auc_score(y, xgb_probabilities)

In [ ]:
roc_auc_xgb

In [ ]:
X_with_xgb_probs

In [ ]:
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_with_xgb_probs, y, test_size=0.3,stratify=y, random_state=42)

In [ ]:
lgb_model = LGBMClassifier(**Lgbm_bparams)
lgb_model.fit(
    X_train2, y_train2,
    
)

In [ ]:
y_pred_stack = lgb_model.predict(X_val2)

# Obtain the classification report
class_report = classification_report(y_val2, y_pred_stack)
print(class_report)

In [ ]:
conf_matrix = confusion_matrix(y_val2, y_pred_stack)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
for model_name, model in zip(classifiers, models + [soft_voting_classifier, hard_voting_classifier]):
    
    predicted_labels = cross_val_predict(model, X, y, cv=kfold)
    
    conf_mat = confusion_matrix(y, predicted_labels)
    
    class_names = [str(i) for i in np.unique(y)]
    
    # Plot confusion matrix
    plot_confusion_matrix(conf_mat, class_names, model_name)

In [ ]:
f, ax = plt.subplots(3, 3, figsize=(12, 10))

for i, clf in enumerate(models):
    y_pred = cross_val_predict(clf, X, y, cv=10)

    cm = confusion_matrix(y, y_pred)

    sns.heatmap(cm, ax=ax[i // 3, i % 3], annot=True, fmt="2.0f")
    ax[i // 3, i % 3].set_title(f"Matrix for {classifiers[i]}")
    ax[i // 3, i % 3].set_xlabel("Predicted")
    ax[i // 3, i % 3].set_ylabel("True")

# Adjust subplot spacing
plt.subplots_adjust(hspace=0.5, wspace=0.5)

# Show the plot
plt.show()

In [ ]:
xgb_model = XGBClassifier(
    scale_pos_weight=11,
    objective='binary:logistic',
    eval_metric='auc',
    reg_lambda=2,
    enable_categorical=True
)
xgb_model.fit(X_train, y_train)

# Step 3: Generate predictions using XGBoost
xgb_preds = xgb_model.predict_proba(X_test)[:, 1]

# Step 4: Add XGBoost predictions as a new feature
X_test_with_xgb_preds = pd.concat([X_test, pd.Series(xgb_preds, name='XGBoost_Predictions')], axis=1)

# Step 5: Train LightGBM model on the updated feature matrix
lgbm_model = LGBMClassifier(
    scale_pos_weight=11,
    objective='binary',
    boosting_type='gbdt',
    metric='auc_pr'
)
lgbm_model.fit(X_test_with_xgb_preds, y_test)

# Now you can evaluate the performance
final_preds = lgbm_model.predict(X_test_with_xgb_preds)
recall_macro = recall_score(y_test, final_preds, average='macro')
print(f"Recall Macro: {recall_macro}")

In [ ]:
lgbm_model.get_params()

In [ ]:
2**5